In [ ]:
import numpy as np
import random
NUM_VMS = 5
NUM_HOSTS = 3
POP_SIZE = 20
GENERATIONS = 100
MUTATION_RATE = 0.1
vm_demands = np.array([
    [20, 50],
    [40, 80],
    [30, 70],
    [60, 100],
    [10, 30]
])
host_resources = np.array([
    [100, 200],
    [150, 250],
    [120, 220]
])
def initialize_population():
    return [np.random.randint(0, NUM_HOSTS, size=NUM_VMS) for _ in range(POP_SIZE)]
def calculate_fitness(chromosome):
    host_usage = np.zeros_like(host_resources)
    penalty = 0
    for vm_index, host_index in enumerate(chromosome):
        cpu_demand, mem_demand = vm_demands[vm_index]
        host_usage[host_index][0] += cpu_demand
        host_usage[host_index][1] += mem_demand
        if (host_usage[host_index][0] > host_resources[host_index][0] or
            host_usage[host_index][1] > host_resources[host_index][1]):
            penalty += 100
    utilization_score = 0
    for i in range(NUM_HOSTS):
        cpu_util = host_usage[i][0] / host_resources[i][0] if host_resources[i][0] else 0
        mem_util = host_usage[i][1] / host_resources[i][1] if host_resources[i][1] else 0
        utilization_score += (cpu_util + mem_util) / 2
    return utilization_score - penalty
def crossover(p1, p2):
    point = random.randint(1, NUM_VMS - 2)
    return np.concatenate((p1[:point], p2[point:]))
def mutate(chrom, rate=MUTATION_RATE):
    for i in range(NUM_VMS):
        if random.random() < rate:
            chrom[i] = random.randint(0, NUM_HOSTS - 1)
    return chrom
population = initialize_population()
for gen in range(GENERATIONS):
    fitness_scores = [calculate_fitness(ind) for ind in population]
    sorted_pop = [x for x in [x for _, x in sorted(zip(fitness_scores, population), key=lambda pair: pair[0], reverse=True)]]
    top_half = sorted_pop[:POP_SIZE // 2]
    offspring = []
    while len(offspring) < POP_SIZE // 2:
        p1, p2 = random.sample(top_half, 2)
        child = crossover(p1, p2)
        child = mutate(child)
        offspring.append(child)
    population = top_half + offspring
best_individual = max(population, key=calculate_fitness)
print("\nBest VM placement (VM → Host):")
for vm_index, host_index in enumerate(best_individual):
    print(f"  VM {vm_index} → Host {host_index}")
host_usage = np.zeros_like(host_resources)
for vm_index, host_index in enumerate(best_individual):
    host_usage[host_index] += vm_demands[vm_index]
print("\nFinal Host Resource Usage:")
for i in range(NUM_HOSTS):
    print(f"  Host {i}: CPU = {host_usage[i][0]}/{host_resources[i][0]}, "
          f"Memory = {host_usage[i][1]}/{host_resources[i][1]}")


Best VM placement (VM → Host):
  VM 0 → Host 2
  VM 1 → Host 2
  VM 2 → Host 0
  VM 3 → Host 0
  VM 4 → Host 0

Final Host Resource Usage:
  Host 0: CPU = 100/100, Memory = 200/200
  Host 1: CPU = 0/150, Memory = 0/250
  Host 2: CPU = 60/120, Memory = 130/220
